In [1]:
import sys
sys.path.append("..") 

In [2]:
# 1. Import libraries
import pandas as pd
import logging

from src.preprocessing import (
    handle_missing_values, clean_data_types, remove_duplicates, save_cleaned_data
)

from src.feature_engineering import (
    add_time_features, add_frequency_features, merge_ip_country, save_feature_engineered_data
)

from src.transformers import (
    get_preprocessor, apply_balancing
)

logging.basicConfig(level=logging.INFO)


ModuleNotFoundError: No module named 'sklearn'

In [12]:
# Setup logging
logging.basicConfig(level=logging.INFO)


In [ ]:
# 2. Load raw data
fraud_df = pd.read_csv("../data/raw/Fraud_Data.csv")
ip_df = pd.read_csv("../data/raw/IpAddress_to_Country.csv")
credit_df = pd.read_csv("../data/raw/creditcard.csv")


In [14]:
#inspect datatypes
print("Fraud Dataset Types:\n", fraud_df.dtypes)
print("\nCredit Dataset Types:\n", credit_df.dtypes)
print("\nIP Dataset Sample:\n", ip_df.head())

Fraud Dataset Types:
 user_id             int64
signup_time        object
purchase_time      object
purchase_value      int64
device_id          object
source             object
browser            object
sex                object
age                 int64
ip_address        float64
class               int64
dtype: object

Credit Dataset Types:
 Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       int64
dtype: object

IP Dataset Sample:
    lower_bound_ip_address  upper_bound_ip_address    country


In [ ]:
# handle missing values
# convert data types
# remove duplicates
fraud_df = handle_missing_values(fraud_df)
fraud_df = clean_data_types(fraud_df, datetime_cols=["signup_time", "purchase_time"])
fraud_df = remove_duplicates(fraud_df)

credit_df = handle_missing_values(credit_df)
credit_df = remove_duplicates(credit_df)

save_cleaned_data(fraud_df, "fraud_data_cleaned.parquet")
save_cleaned_data(credit_df, "creditcard_data_cleaned.parquet")


INFO:root:Original shape: (151112, 11)
INFO:root:Shape after dropping columns with >50.0% missing: (151112, 11)
INFO:root:Removed 0 duplicate rows


In [ ]:
# eda
import seaborn as sns
import matplotlib.pyplot as plt

# Class Imbalance Check
sns.countplot(x='class', data=fraud_df)
plt.title('E-Commerce Fraud Class Distribution')

sns.countplot(x='Class', data=credit_df)
plt.title('Bank Transaction Fraud Class Distribution')


In [ ]:
# geolocation merge with ip
fraud_df = merge_ip_country(fraud_df, ip_df)
save_cleaned_data(fraud_df, "fraud_data_with_country.parquet")


In [ ]:
# feature engineering
fraud_df = add_time_features(fraud_df)
fraud_df = add_frequency_features(fraud_df)

save_feature_engineered_data(fraud_df, "fraud_data_features.parquet")



KeyboardInterrupt: 

In [ ]:
# 4. Feature Engineering
fraud_df = add_time_features(fraud_df)
fraud_df = add_frequency_features(fraud_df)
fraud_df = merge_ip_country(fraud_df, ip_df)


In [9]:
# Extract hour_of_day and day_of_week from purchase_time
fraud_df['hour_of_day'] = fraud_df['purchase_time'].dt.hour
fraud_df['day_of_week'] = fraud_df['purchase_time'].dt.dayofweek


In [10]:
# Example: transaction counts per user and per device (within fraud_df)
fraud_df['user_transaction_count'] = fraud_df.groupby('user_id')['purchase_time'].transform('count')
fraud_df['device_transaction_count'] = fraud_df.groupby('device_id')['purchase_time'].transform('count')


In [11]:
# 5. Save cleaned and feature-engineered data
os.makedirs("../data/processed", exist_ok=True)

In [12]:
fraud_df.to_parquet("../data/processed/fraud_data_features.parquet", index=False)
print("Saved feature-engineered fraud data to ../data/processed/fraud_data_features.parquet")


ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:
credit_df.to_parquet("../data/processed/credit_data_features.parquet", index=False)
print("Saved cleaned credit data to ../data/processed/credit_data_features.parquet")